# patsy 패키지
patsy 패키지를 사용하여 데이터프레임에서 원하는 데이터만 선택하거나 새로운 데이터를 조합 생성하는 방법<br>
patsy 패키지가 제공하는 `demo_data()` 함수 사용
* `demo_data()` 함수는 x로 시작하는 변수에 대해 임의의 실수 데이터를 생성

In [104]:
from patsy import demo_data
df = pd.DataFrame(demo_data('x1', 'x2','x3','x4','x5'))
df

,x1,x2,x3,x4,x5
0,1.764052,-0.977278,0.144044,0.333674,-2.552990
1,0.400157,0.950088,1.454274,1.494079,0.653619
2,0.978738,-0.151357,0.761038,-0.205158,0.864436
3,2.240893,-0.103219,0.121675,0.313068,-0.742165
4,1.867558,0.410599,0.443863,-0.854096,2.269755


### dmatrix()

* `dmatrix()` 함수는 데이터 프레임에 상수항을 추가하거나 원하는 데이터만 선택하거나 변형할 수 있다. 

`dmatrx()` 함수에 **모형 정의 문자열** formula와 원 데이터를 담은 **데이터프레임** data을 입력하면<br> formula에서 지정한 대로 변환된 데이터 data_transformed를 출력한다.

```data_transformed = dmatrix(formula, data)```

* formula : 데이터 열 이름 기반으로 구성된 문자열
* 선택하고자 하는 데이터 열 이름을 + 로 연결한 formula 문자열을 입력하면 자동으로 해당 데이터만 출력
* ex) 전체 데이터 중 x1만을 뽑고 싶다면 formula 문자열에 x1 + 0 이라고 입력한다.

In [108]:
from patsy import dmatrix 

dmatrix("x1 + 0", data=df)

DesignMatrix with shape (5, 1)
       x1
  1.76405
  0.40016
  0.97874
  2.24089
  1.86756
  Terms:
    'x1' (column 0)

In [109]:
# 전체 데이터 중 x1, x2, x3를 뽑고 싶으면 formula 문자열에 x1 + x2 + x3 + 0이라고 입력
dmatrix('x1 + x2 + x3 + 0', data = df)

DesignMatrix with shape (5, 3)
       x1        x2       x3
  1.76405  -0.97728  0.14404
  0.40016   0.95009  1.45427
  0.97874  -0.15136  0.76104
  2.24089  -0.10322  0.12168
  1.86756   0.41060  0.44386
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x3' (column 2)

* **`+0`** 은 1로 구성된 상수항을 포함하지 말라는 의미이다.
* **`- 1`** 도 + 0과 같은 뜻
* 만약 이 부분이 없으면 patsy는 자동으로 가장 앞에 `Intercept`란 이름의 **상수항 데이터**를 추가함

In [110]:
dmatrix('x1+ x2 + x3', data = df)

DesignMatrix with shape (5, 4)
  Intercept       x1        x2       x3
          1  1.76405  -0.97728  0.14404
          1  0.40016   0.95009  1.45427
          1  0.97874  -0.15136  0.76104
          1  2.24089  -0.10322  0.12168
          1  1.86756   0.41060  0.44386
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'x2' (column 2)
    'x3' (column 3)

`dmatrix()` 함수는 변수를 어떤 함수에 넣어서 다른 값으로 만드는 수학 변환(transform)도 가능

In [112]:
dmatrix('x1 + np.log(np.abs(x2))', df)

DesignMatrix with shape (5, 3)
  Intercept       x1  np.log(np.abs(x2))
          1  1.76405            -0.02298
          1  0.40016            -0.05120
          1  0.97874            -1.88811
          1  2.24089            -2.27090
          1  1.86756            -0.89014
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'np.log(np.abs(x2))' (column 2)

numpy 함수 뿐 아니라 `사용자 정의 함수`도 사용 가능

In [114]:
def ten_times(x):
    return 10 * x

In [115]:
dmatrix("ten_times(x1)", df)

DesignMatrix with shape (5, 2)
  Intercept  ten_times(x1)
          1       17.64052
          1        4.00157
          1        9.78738
          1       22.40893
          1       18.67558
  Terms:
    'Intercept' (column 0)
    'ten_times(x1)' (column 1)

* 기존의 데이터를 조합 연산하여 새로운 데이터를 만드는 것도 가능
* 특히 두 변수를 곱해서 만들어지는 새로운 변수를 **상호작용(interaction)** 이라고 한다.
    * 상호작용은 **`:`** 기호를 사용하여 만들어진다.

In [116]:
dmatrix("x1 + x2 + x1:x2 + 0", df)

DesignMatrix with shape (5, 3)
       x1        x2     x1:x2
  1.76405  -0.97728  -1.72397
  0.40016   0.95009   0.38018
  0.97874  -0.15136  -0.14814
  2.24089  -0.10322  -0.23130
  1.86756   0.41060   0.76682
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

* 두 변수와 상호작용을 한꺼번에 표시할 때는 **`*`** 를 사용한다.
* 따라서 다음 식은 바로 전의 수식과 동일하다.

In [117]:
dmatrix("x1 * x2 + 0", df)

DesignMatrix with shape (5, 3)
       x1        x2     x1:x2
  1.76405  -0.97728  -1.72397
  0.40016   0.95009   0.38018
  0.97874  -0.15136  -0.14814
  2.24089  -0.10322  -0.23130
  1.86756   0.41060   0.76682
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

* 상호작용을 제외한 경우에는 I()라는 연산자를 사용하여 연산과정을 명시해야 한다. 

In [118]:
# 두 변수 x1과 x2를 더하여 새로운 데이터를 만들고 싶을 때
dmatrix("x1 + x2 + I(x1 + x2) + 0", df)

DesignMatrix with shape (5, 3)
       x1        x2  I(x1 + x2)
  1.76405  -0.97728     0.78677
  0.40016   0.95009     1.35025
  0.97874  -0.15136     0.82738
  2.24089  -0.10322     2.13767
  1.86756   0.41060     2.27816
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'I(x1 + x2)' (column 2)

선형회귀분석을 할 때는 `조건수(condition number)`의 영향 때문에 <br>데이터의 **평균을 0으로 표준편차를 1**로 만드는 `스케일링(scaling)` 작업을 하는 것이 분석 결과의 품질을 높일 수 있다. `patsy` 패키지는 스케일링을 위한 함수도 제공한다.
* `center()` : 평균을 0으로 스케일링

* `standardize()` : 평균을 0으로하고 표준편차를 1로 스케일링

* `scale()` : standardize() 과 같음

In [119]:
dm = dmatrix("center(x1) + 0", df)
dm

DesignMatrix with shape (5, 1)
  center(x1)
     0.31377
    -1.05012
    -0.47154
     0.79061
     0.41728
  Terms:
    'center(x1)' (column 0)

In [120]:
# x1 데이터에서 x1의 평균을 빼는 것
df.x1 - np.mean(df.x1)

0    0.313773
1   -1.050123
2   -0.471542
3    0.790613
4    0.417278
Name: x1, dtype: float64

* 이 때 평균값은 `design_info`라는 속성 내부에 저장된다.
* 값을 저장하는 이유는 새로운 검증용 데이터로 실제 예측을 할 때 <br>모형을 학습할 때 사용한 것과 동일한 전처리를 해야 한다. <br>이 때 학습용 데이터의 평균값을 빼야 한다는 점에서 처리 과정에 계산한 평균값을 기억하고 있어야 한다.

In [138]:
dm.design_info.factor_infos

{EvalFactor('center(x1)'): FactorInfo(factor=EvalFactor('center(x1)'),
            type='numerical',
            state=<factor state>,
            num_columns=1)}